In [1]:
import torch._C
import torch
import torch.serialization
# from torch import nn
# import os
# import matplotlib.pyplot as plt
# from PIL import Image
# from glob import glob
import onnx
import onnxruntime as rt
import numpy as np
from model.parseq import PARSeq
from omegaconf import DictConfig, open_dict, OmegaConf

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# args
OUTPUT_FILE = 'parseq_ar_r1.onnx'
device = torch.device('cuda:2')
ckpt_path = 'outputs/exp_logs_baseline_all/Baseline_best_parseq_ckpt.pth'
config_path = 'outputs/exp_logs_baseline_all/config.yaml'

# config load
config = OmegaConf.load(config_path)
# config.model.decode_ar = False
# config.model.decode_ar = False
# model load 
model = PARSeq(**config.model)
# h, w = config.model.img_size
# model = torch.jit.script(model, example_inputs={model: torch.randn(1, 3, h, w, requires_grad=True)})

model.load_state_dict(torch.load(ckpt_path)['model'])
model._device = torch.device(device)
model = model.eval().to(device)
# print('Model parameters: ', config.model, sep='\n')

# input define
h, w = config.model.img_size
x = torch.randn(1, 3, h, w, requires_grad=True)
x = x.to(device)

# input test results
out = model(x)

#Onnx export details
dynamic_axes = None
dynamic_export = False
opset_version = 14
show = False
if dynamic_export:
    dynamic_axes = {
        'input': {
            0: 'batch',
#             1: 'channel',
#             2: 'height',
#             3: 'widht'
        },
        'output': {
            0: 'batch',
            1: 'max_len',
#             2: 'charset_len'
        }
    }

with torch.no_grad():
    torch.onnx.export(
        model, x,
        OUTPUT_FILE,
        training=None,
        input_names=['input'],
        output_names=['output'],
        export_params=True,
        verbose=show,
        opset_version=opset_version,
        dynamic_axes=dynamic_axes)
    print(f'Successfully exported ONNX model: {OUTPUT_FILE}')

# helper function
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

## Onnx and torch model output similarity check

# Onnx model loading
onnx_model = onnx.load(OUTPUT_FILE)
onnx.checker.check_model(onnx_model)

ort_session = rt.InferenceSession(OUTPUT_FILE)

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(out), ort_outs[0], rtol=1e-05, atol=1e-01)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

FileNotFoundError: [Errno 2] No such file or directory: 'outputs/exp_logs_baseline_all/Baseline_best_parseq_ckpt.pth'

In [1]:
# ## general testing
# for _ in range(10):
#     x = torch.randn(1, 3, h, w, requires_grad=True)
#     x = x.to(device)

#     out = model(x)
#     x = to_numpy(x)
#     # compute ONNX Runtime output prediction
#     ort_inputs = {ort_session.get_inputs()[0].name: x}
#     ort_outs = ort_session.run(['output'], ort_inputs)

#     # compare ONNX Runtime and PyTorch results
# #     np.testing.assert_allclose(to_numpy(out), ort_outs[0], rtol=1e-05, atol=1e-01)
#     print(torch.abs(out.detach().cpu() - torch.tensor(ort_outs[0])).shape)
#     print(torch.max(torch.abs(out.detach().cpu() - torch.tensor(ort_outs[0]))))
#     print("Exported model has been tested with ONNXRuntime, and the result looks good!")  

In [2]:
%%timeit

out = model(x)

NameError: name 'model' is not defined

In [6]:
config.model.decode_ar

True

In [8]:
out.shape

torch.Size([1, 26, 1794])